# Basic Raytracing

__raytracing pipeline, raytracing manager, build and update ADS__


In [ ]:
import vulky as vk
import numpy as np

vk.create_device(debug=False)

SCREEN_WIDTH = 512
SCREEN_HEIGHT = 512

render_target = vk.image_2D(
    image_format=vk.Format.VEC4,
    width=SCREEN_WIDTH,
    height=SCREEN_HEIGHT,
    mips=1,  # Important to make the image single resource
    usage=vk.ImageUsage.STORAGE,
    memory=vk.MemoryLocation.GPU
)

data = vk.load_obj('./vulky_data/bunny.obj')
vertices_data, indices_data = vk.create_mesh(data, mode='po')

In [ ]:
triangles_vb = vk.structured_buffer(
    count=len(vertices_data), 
    element_description=dict(
        P=vk.vec3,
        N=vk.vec3,
        C=vk.vec2
    ),
    usage=vk.BufferUsage.RAYTRACING_RESOURCE    
).load(vertices_data)
triangles_ib = vk.structured_buffer(
    count=indices_data.numel(),
    element_description=int,
    usage=vk.BufferUsage.RAYTRACING_RESOURCE
).load(indices_data)

triangles = vk.triangle_collection()
triangles.append(triangles_vb, triangles_ib)
geometry_ads = vk.ads_model(triangles)

instance_transforms = vk.tensor(3, 4, 3)
for i in range(3):
    instance_transforms[i] = vk.mat4x3.trs(vk.vec3(np.cos(i * 6.28 / 3), -0.2, np.sin(i * 6.28 / 3)), vk.vec3(0., 1., 0.0), i*3.14159/3, vk.vec3.one())
    
instances = vk.instance_buffer(3, memory=vk.MemoryLocation.CPU)  # intentionally on the CPU for fast update
with instances.map(mode='inout') as m:
    m.transform = vk.mat4x3(instance_transforms).transposed()
    m.accelerationStructureReference = geometry_ads.handle  # All geometries are the same
scene_ads = vk.ads_scene(instances)    

scratch_buffer = vk.scratch_buffer(geometry_ads, scene_ads)
    
with vk.raytracing_manager() as man:
    man.build_ads(geometry_ads, scratch_buffer)
    man.build_ads(scene_ads, scratch_buffer)

In [ ]:
global_transform = vk.object_buffer(
    vk.Layout.from_structure(
        vk.LayoutAlignment.SCALAR,
        Proj=vk.mat4,
        View=vk.mat4
    )
)

with global_transform as b:
    b.Proj = vk.mat4.perspective()
    b.View = vk.mat4.look_at(vk.vec3(0., 1., 4.), vk.vec3(0., 0.5, 0.), vk.vec3(0.,1., 0.))

In [ ]:
ray_generation_shader_code = """
#version 460
#extension GL_EXT_ray_tracing : require

layout(set = 0, binding = 0) uniform accelerationStructureEXT Scene;
layout(set = 0, binding = 1) uniform Globals {
    mat4 proj;
    mat4 view;
} globals;
layout(set = 0, binding = 2, rgba32f) uniform image2D ResultImage;

layout(location = 0) rayPayloadEXT vec4 ResultColor;

void main() {
    vec2 uv = (gl_LaunchIDEXT.xy + vec2(0.5)) / vec2(gl_LaunchSizeEXT.xy);
    vec4 P = vec4(uv.x * 2 - 1.0, 1.0 - uv.y * 2, 0.0, 1.0);
    vec4 T = vec4(uv.x * 2 - 1.0, 1.0 - uv.y * 2, 1.0, 1.0);
    mat4 inv_transform = inverse(globals.proj * globals.view); 
    P = inv_transform * P;
    T = inv_transform * T;
    vec3 o = P.xyz / P.w;
    vec3 d = normalize(T.xyz/T.w - o);
    traceRayEXT(Scene, gl_RayFlagsNoneEXT, 0xFF, 0, 0, 0, o, 0.001, d, 1000.0f, 0);
    imageStore(ResultImage, ivec2(gl_LaunchIDEXT.xy), ResultColor);
}  
"""
closest_hit_shader_code = """
#version 460
#extension GL_EXT_ray_tracing : require
#extension GL_EXT_scalar_block_layout : require

layout(location = 0) rayPayloadInEXT vec4 ResultColor;
hitAttributeEXT vec2 HitAttribs;

struct Vertex {
    vec3 P;
    vec3 N;
    vec2 C;
};

layout(scalar, set=0, binding = 3) readonly buffer Vertices {
    Vertex data[];
} vertices;

layout(scalar, set=0, binding = 4) readonly buffer Indices {
    int data[];
} indices;

void main() {
    int triangleIndex = gl_PrimitiveID;
    Vertex v0 = vertices.data[indices.data[triangleIndex*3 + 0]];
    Vertex v1 = vertices.data[indices.data[triangleIndex*3 + 1]];
    Vertex v2 = vertices.data[indices.data[triangleIndex*3 + 2]];
    vec3 coord = vec3(1 - HitAttribs.x - HitAttribs.y, HitAttribs.x, HitAttribs.y);
    vec3 N =  v0.N * coord.x + v1.N * coord.y + v2.N * coord.z;
    N = normalize(gl_ObjectToWorldEXT * vec4(N, 0));
    //ResultColor = vec4(N.x, N.y, N.z, 1);
    vec3 light = vec3(3,3,0.3)/3.14159*max(0.1, dot(N, normalize(vec3(2,1,5))));
    ResultColor = vec4(light.x, light.y, light.z, 1);
}
"""
miss_shader_code = """
#version 460
#extension GL_EXT_ray_tracing : require

layout(location = 0) rayPayloadInEXT vec4 ResultColor;

void main() {
    ResultColor = vec4(0.1, 0.1, 0.7, 1.0);
}  
"""

In [ ]:
pipeline = vk.pipeline_raytracing()
pipeline.layout(set=0, binding=0, scene=vk.DescriptorType.SCENE_ADS)
pipeline.layout(set=0, binding=1, global_transform=vk.DescriptorType.UNIFORM_BUFFER)
pipeline.layout(set=0, binding=2, output_image=vk.DescriptorType.STORAGE_IMAGE)
pipeline.layout(set=0, binding=3, vertices=vk.DescriptorType.STORAGE_BUFFER)
pipeline.layout(set=0, binding=4, indices=vk.DescriptorType.STORAGE_BUFFER)
with pipeline.shader_stages(vk.ShaderStage.RT_GENERATION):
    gen_shader = pipeline.load_shader_from_source(ray_generation_shader_code)
with pipeline.shader_stages(vk.ShaderStage.RT_MISS):
    miss_shader = pipeline.load_shader_from_source(miss_shader_code)
with pipeline.shader_stages(vk.ShaderStage.RT_CLOSEST_HIT):
    closest_hit_shader = pipeline.load_shader_from_source(closest_hit_shader_code)
gen_group = pipeline.create_rt_gen_group(gen_shader)
hit_group = pipeline.create_rt_hit_group(closest_hit=closest_hit_shader)
miss_group = pipeline.create_rt_miss_group(miss_shader)
pipeline.close()

In [ ]:
rt_program = pipeline.create_rt_program(1, 1, 1, 1)
with rt_program:
    rt_program.set_generation(0, gen_group)
    rt_program.set_hit_group(0, hit_group)
    rt_program.set_miss(0, miss_group)

In [ ]:
global_bindings = pipeline.create_descriptor_set_collection(set=0, count=1)
global_bindings[0].update(
    scene = scene_ads,
    global_transform = global_transform,
    output_image = render_target,
    vertices = triangles_vb,
    indices = triangles_ib
)

In [ ]:
man = vk.raytracing_manager()
man.set_pipeline(pipeline)
man.bind(global_bindings[0])
man.dispatch_rays(rt_program, SCREEN_WIDTH, SCREEN_HEIGHT)
man.freeze()

In [ ]:
vk.submit(man)

In [ ]:
staging = vk.tensor(render_target.height, render_target.width, 4)
render_target.save(staging)
import matplotlib.pyplot as plt
plt.imshow(staging.cpu() ** (1.0/2.2))
plt.gca().axis('off')
plt.tight_layout(pad=0.0)
plt.show()

Let's create the video

In [ ]:
video_data = vk.tensor(100, SCREEN_HEIGHT, SCREEN_WIDTH, 3)  # 100 frames

In [ ]:
for i in range(len(video_data)):
    alpha = i / len(video_data)
    look_at = vk.vec3(np.cos(alpha * 7), np.sin(alpha)*0.1 + 0.2, np.sin(alpha * 7))*(5 + np.sin(alpha))
    # we dont need to map because is permanently mapped and update is always CPU -> GPU
    with global_transform as b:
        b.View = vk.mat4.look_at(look_at, vk.vec3(0., 0., 0.), vk.vec3(0.,1., 0.))
    # update instance transforms for the top-level    
    for j in range(3):
        instance_transforms[j] = vk.mat4x3.trs(vk.vec3(np.cos(j * 6.28 / 3), -0.2, np.sin(j * 
          6.28 / 3)), vk.vec3(0., 1., 0.0), j*3.14159/3 + alpha * 6.28 * 2, vk.vec3.one())
    # the scene ads is bound to this instance buffer already
    with instances.map(mode='inout') as m:
        m.transform = vk.mat4x3(instance_transforms).transposed()
    with vk.raytracing_manager() as b:
        b.update_ads(scene_ads, scratch_buffer)  # update top-level ads every frame
    vk.submit(man)
    render_target.save(staging)
    video_data[i] = staging[...,:3] ** (1.0/2.2)  # copy current frame to video (only RGB)   

In [ ]:
vk.save_video(video_data, 'teaser7.webp', 10, quality=100)
import moviepy.editor
moviepy.editor.ipython_display("teaser7.webp", filetype='image')